In [1]:
import os
import tweepy

In [2]:
# Authenticate app context
auth = tweepy.OAuthHandler(os.environ['TWITTER_API_KEY'], os.environ['TWITTER_API_SECRET'])
auth.set_access_token(os.environ['TWITTER_ACCESS_TOKEN'], os.environ['TWITTER_ACCESS_TOKEN_SECRET'])
api = tweepy.API(auth)

In [3]:
# Get follower ids
screen_name = 'francopiccolo'
follower_ids = api.get_follower_ids(screen_name=screen_name)
print(len(follower_ids))

181


In [28]:
# Get followers from list of user objects
screen_name = 'francopiccolo'
cursor = -1
followers = []
while cursor:
    users, cursors = api.get_followers(screen_name=screen_name, cursor=cursor)
    for user in users:
        followers.append(user.screen_name)
    cursor = cursors[1]

In [4]:
# Get friend ids
screen_name = 'francopiccolo'
friend_ids = api.get_friend_ids(screen_name=screen_name)
print(len(friend_ids))

1341


In [5]:
# Get users from list of user ids
def get_users(user_ids, chunk_size=100):
    num_users = len(user_ids)
    start_idx = 0
    end_idx = chunk_size
    users = []
    keep = True
    while keep:
        if end_idx > num_users:
            end_idx = num_users
            keep = False
        users.extend(api.lookup_users(user_id=user_ids[start_idx:end_idx]))
        start_idx += chunk_size
        end_idx += chunk_size
    return users

In [4]:
# Unfollow small accounts not following me
user_ids_to_unfollow = [user for user in friend_ids if user not in follower_ids]
users_to_unfollow = get_users(user_ids_to_unfollow)
users_to_unfollow = [user for user in users_to_unfollow if user.followers_count < 1000]
for user in users_to_unfollow:
    api.destroy_friendship(screen_name=user.screen_name)